In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os
if __name__=='__main__':
    path_of_raw_data=input('path of 21D raw data:')
    path_of_cus_raw_data=input('path of customer info raw data:')
    df=pd.read_csv(path_of_raw_data)
    df_cusinfo=pd.read_csv(path_of_cus_raw_data)
    df_cusinfo['AvgMonReport_R3M']=df_cusinfo['AvgMonReport_R3M'].fillna(0)
    path=input('Where to save output?')

    companylist_ID=[]
    companyresult21days=[]
    country=[]
    avg_monthly_reports_after=[]

    i=0
    for x in df['User ID']:
        if i == 0:
            companylist_ID.append(df['User ID'][i])
            companyresult21days.append(df['Unnamed: 21'][i+4])
            country.append(df['Country'][i])
            alist = df.loc[i + 9]
            length = len(list(alist.dropna())) - 1
            accu_reports = list(alist.dropna())[-1]
            accu_reports = int(accu_reports)
            avg_monthly_reports_after.append(accu_reports / length * 30)
        elif x== 'User ID':
            companylist_ID.append(df['User ID'][i+1])
            companyresult21days.append(df['Unnamed: 21'][i+5])
            country.append(df['Country'][i+1])
            alist = df.loc[i + 10]
            length = len(list(alist.dropna())) - 1
            accu_reports = list(alist.dropna())[-1]
            accu_reports = int(accu_reports)
            avg_monthly_reports_after.append(accu_reports / length * 30)
        i=i+1
    

    companyresult=pd.DataFrame()

    companyresult['UserID']=companylist_ID
    name=[]
    R3Mlist=[]
    for id in companyresult['UserID']:
            if len(df_cusinfo.loc[df_cusinfo['UserID'] == int(id)]['UserName'].values) == 0:
                char_name = '/'
                R3M='/'
            elif len(df_cusinfo.loc[df_cusinfo['UserID'] == int(id)]['UserName'].values) > 0:
                char_name = df_cusinfo.loc[df_cusinfo['UserID'] == int(id)]['UserName'].values[0]
                R3M=list(df_cusinfo.loc[df_cusinfo['UserID'] == int(id)]['AvgMonReport_R3M'])[0]
             
            name.append(char_name)
            R3Mlist.append(R3M)


    companyresult['User Name']=name
    companyresult['no. of reports before 21D(monthly)']=R3Mlist
    companyresult['no. of reports during 21D']=companyresult21days
    companyresult['no. of reports during 21D']=pd.to_numeric(companyresult['no. of reports during 21D'])
    companyresult['no. of reports during 21D(projected monthly)']=pd.to_numeric(companyresult['no. of reports during 21D'])*1.4
    companyresult['no. of reports after 21D(projected monthly)']=avg_monthly_reports_after
    companyresult['Country']=country
    companyresult.loc[companyresult['no. of reports during 21D']>=21,'Successful completed?']='Successful'
    companyresult.loc[companyresult['no. of reports during 21D']<21,'Successful completed?']='Unsuccessful'
  

    success_rate=pd.DataFrame(companyresult['Successful completed?'].value_counts())
    plt.pie(success_rate['Successful completed?'],labels=success_rate.index,autopct='%1.0f%%',labeldistance=1.1,shadow=True)
    plt.title(label="Success rate: Overall",pad=1.0)
    plt.savefig(os.path.join(path,'overall_success_rate.png'))
    plt.clf()

    countrylist=companyresult['Country'].unique()
    for country in countrylist:
    companyresult_by_country=companyresult[companyresult['Country']==country]
    success_rate_by_country=companyresult_by_country['Successful completed?'].value_counts()
    plt.pie(success_rate_by_country, labels= success_rate_by_country.index, autopct='%1.0f%%', labeldistance=1.1,
            shadow=True)
    plt.title(label="Success rate: {}".format(country), pad=1.0)
    plt.savefig(os.path.join(path,'{}_success_rate.png'.format(country)))
    plt.clf()


    companyresult_olduser=companyresult[companyresult['User Name']!='/']
    result_in_three_period= {}
    result_in_three_period['Before 21D']=round(companyresult_olduser['no. of reports before 21D(monthly)'].mean(),2)
    result_in_three_period['During 21D']=round(companyresult_olduser['no. of reports during 21D(projected monthly)'].mean(),2)
    result_in_three_period['After 21D']=round(companyresult_olduser['no. of reports after 21D(projected monthly)'].mean(),2)
    plt.bar(list(result_in_three_period.keys()),list(result_in_three_period.values()), width = 0.4)
    plt.ylabel("Average no. of monthly reports")
    plt.xlabel("Period")
    plt.title("Average no. of monthly reports by period:Overall")
    for i in range(len(list(result_in_three_period.values()))):
        plt.text(i,list(result_in_three_period.values())[i],str(list(result_in_three_period.values())[i]),ha='center',va='bottom')
    plt.savefig(os.path.join(path,'Overall_no_of_reports_change.png'))
    plt.clf()
    for country in countrylist:
        companyresult_olduser_by_country = companyresult_olduser[companyresult_olduser['Country'] == country]
        result_in_three_period = {}
        result_in_three_period['Before 21D'] = round(companyresult_olduser_by_country['no. of reports before 21D(monthly)'].mean(), 2)
        result_in_three_period['During 21D'] = round(companyresult_olduser_by_country['no. of reports during 21D(projected monthly)'].mean(), 2)
        result_in_three_period['After 21D'] = round(companyresult_olduser_by_country['no. of reports after 21D(projected monthly)'].mean(), 2)
        plt.bar(list(result_in_three_period.keys()), list(result_in_three_period.values()), width=0.4)
        plt.ylabel("Average no. of monthly reports")
        plt.xlabel("Period")
        plt.title("Average no. of monthly reports by period:{}".format(country))
        for i in range(len(list(result_in_three_period.values()))):
            plt.text(i, list(result_in_three_period.values())[i], str(list(result_in_three_period.values())[i]),
                    ha='center', va='bottom')
        plt.savefig(os.path.join(path, '{}_no_of_reports_change.png'.format(country)))
        plt.clf()

    # companyresult.to_excel(os.path.join(path,'21D_individual_performance.xlsx'),index=False,sheet_name='sheet1')

    overall_performance={}

    overall_performance['overall']={
        'Number of comparable hospital':len(companyresult_olduser),
        'Avg no. of reports before 21D':companyresult_olduser['no. of reports before 21D(monthly)'].mean(),
        'Avg no. of reports during 21D':companyresult_olduser['no. of reports during 21D(projected monthly)'].mean(),
        'Avg no. of reports after 21D':companyresult_olduser['no. of reports after 21D(projected monthly)'].mean(),
        'Percantage change before-during':(companyresult_olduser['no. of reports during 21D(projected monthly)'].mean()-companyresult_olduser['no. of reports before 21D(monthly)'].mean())*100/companyresult_olduser['no. of reports before 21D(monthly)'].mean(),
        'Percantage change before-after':(companyresult_olduser['no. of reports after 21D(projected monthly)'].mean()-companyresult_olduser['no. of reports before 21D(monthly)'].mean())*100/companyresult_olduser['no. of reports before 21D(monthly)'].mean(),
    }

    all_companyresult_olduser=companyresult_olduser
    for country in countrylist:
        companyresult_olduser=all_companyresult_olduser[all_companyresult_olduser['Country']==country]
        overall_performance[country] = {
            'Number of comparable hospital': len(companyresult_olduser),
            'Avg no. of reports before 21D': companyresult_olduser['no. of reports before 21D(monthly)'].mean(),
            'Avg no. of reports during 21D': companyresult_olduser['no. of reports during 21D(projected monthly)'].mean(),
            'Avg no. of reports after 21D': companyresult_olduser['no. of reports after 21D(projected monthly)'].mean(),
            'Percantage change before-during': str((companyresult_olduser[
                                                    'no. of reports during 21D(projected monthly)'].mean() -
                                                companyresult_olduser['no. of reports before 21D(monthly)'].mean()) * 100 /
                                            companyresult_olduser['no. of reports before 21D(monthly)'].mean())+'%',
            'Percantage change before-after': str((companyresult_olduser['no. of reports after 21D(projected monthly)'].mean() -
                                            companyresult_olduser['no. of reports before 21D(monthly)'].mean()) * 100 /
                                            companyresult_olduser['no. of reports before 21D(monthly)'].mean())+'%',
        }


    overall_performance=pd.DataFrame(overall_performance)

    writer = pd.ExcelWriter(os.path.join(path,'21D_performance.xlsx'), engine='xlsxwriter')

    companyresult.to_excel(writer, sheet_name='individual_performance',index=False)
    overall_performance.to_excel(writer, sheet_name='overall_performance')
    writer.save()




path of 21D raw data:C:\Users\yhy\PycharmProjects\workproject\(complete)21D customer usage_1221 - 21D customer usage_1231.csv
path of customer info raw data:C:\Users\yhy\PycharmProjects\workproject\21D customer usage_1221 - Sheet3.csv
Where to save output?C:\Users\yhy\Documents\winterintern\savepic
Results are ready!


<Figure size 432x288 with 0 Axes>